In [1]:
import numpy as np
from glass.g.fmrst import rst_to_array
from glass.fm import tbl_to_obj
from osgeo import gdal
from glass.g.prop.rst import get_cell_idx, get_cell_coord
import math

In [2]:
inShp = r'D:\tst_vis\pnt_tst.shp'
dem = r'D:\tst_vis\dem_test_100.tif'
lulc = r'D:\tst_vis\lulc_test_100.tif'

pnt_df = tbl_to_obj(inShp)
pnt_df['x'] = pnt_df.geometry.x
pnt_df['y'] = pnt_df.geometry.y
pnt = pnt_df.iloc[0]

In [3]:
# Convert data to Py Object
dem_obj  = gdal.Open(dem)
lulc_obj = gdal.Open(lulc)

np_dem  = dem_obj.ReadAsArray()
np_lulc = lulc_obj.ReadAsArray()

In [4]:
# NoData As Nan in both Images

dem_nd = dem_obj.GetRasterBand(1).GetNoDataValue()
lulc_nd = lulc_obj.GetRasterBand(1).GetNoDataValue()

np_dem[np_dem == dem_nd] = np.nan
np_lulc[np_lulc == dem_nd] = np.nan

In [5]:
# Get Rows and Columns Number

lnhs = int(dem_obj.RasterYSize)
cols = int(dem_obj.RasterXSize)

# Get Extent
left, cellx, z, top, c, celly = dem_obj.GetGeoTransform()

right = left + (cols * cellx)
bottom = top - (lnhs - abs(celly))

In [6]:
# Obtain row and column idx

row_idx, col_idx = get_cell_idx(np_dem, pnt.x, pnt.y, left, top, cellx, abs(celly))
print(row_idx)
print(col_idx)

274
123


In [7]:
# Check for data, if not data in lulc and dem, no calculation is required

if np.isnan(np_lulc[row_idx, col_idx]) or np.isnan(np_dem[row_idx, col_idx]):
    print("NoCalculations are necessary")

In [8]:
# Check if observation point is class
if np_lulc[row_idx, col_idx] == 1:
    print("Observation is in class")

In [9]:
def check_visibility(obsrow, obscol, seenrow, seencol, dem, lin, col):
    # Get Observation coordinates
    yobs=obsrow - 0.5
    xobs=obscol - 0.5
    
    # Get Coordinates of test place
    #x_seen, y_seen = get_cell_coord(seenrow, seencol, x_min, y_max, cell_x, cell_y)
    x_seen = seencol - 0.5
    y_seen = seenrow - 0.5
    
    # Get DEM Values for both points
    z_obs  = dem[obsrow, obscol]
    z_seen = dem[seenrow, seencol]
    
    # Vetor diretor da reta no espaco
    vetor_diretor = np.array([[x_seen - xobs, y_seen - yobs, z_seen - z_obs]])
    
    """
    Declive ('m') e ordenada na origem ('b') da equao da reta no pano que une 
    o ponto de observao e o ponto a observar.
    """
    m = np.divide((y_seen - yobs), (x_seen - xobs))
    b = yobs - m * xobs
    
    """
    Pretende-se obter os valores de x e y de intersecao do segmento que une o 
    ponto de observao com o ponto a observar (no plano) com os limites das celulas.
    Para isso, comeca-se no valor minimo de x, e usa-se o menor numero inteiro maior
    que o minimo de x, e vai-se somando 1 ate chegar ao maior inteiro menor que o valor 
    maximo de x. No final inclui-se tambem o as coordenadas do ponto do ultimo
    ponto, que pode ser o ponto de origem ou destino (dependendo de qual dos
    pontos tem maior valor de x), portanto sera
    Faz-se da mesma forma para y.
    """
    seg_x = np.array([np.append(
        np.arange(math.ceil(min(xobs,x_seen)), math.floor(max(xobs,x_seen))+1 , 1),
        max(xobs,x_seen)
    )])
    
    seg_y=np.array([np.append(
        np.arange(math.ceil(min(yobs,y_seen)), math.floor(max(yobs,y_seen))+1 , 1),
        max(yobs,y_seen))])
    
    """
    Para cada valor de x encontrado calcula-se o y, usando a equao da
    reta no plano. Da mesma forma para cada valor de y encontrado calcula-se o valor de
    x
    """
    if x_seen == xobs:
        x1=np.tile(seg_x,(seg_y.shape[0],seg_y.shape[1]))
        #seg_x=np.array([])
        #y1=np.array([])
    elif m == 0:
        y1=m*seg_x + b
        #x1=np.array([])
        #seg_y=np.array([])
    else:
        y1=m*seg_x + b
        x1=(seg_y - b) / m
    
    """
    Constroi-se um vetor para agregar os valores de y aos valores de x
    encontrados acima, e vice versa (os valores de x para os valores de y
    encontrados acima). Isto da-nos as coordenadas dos pontos de intersecao
    dos segmentos com os limites das celulas.
    """
    
    if x_seen != xobs:
        vec1=np.append(seg_x.T, y1.T, axis=1)
    
    if m != 0:
        vec2=np.append(x1.T, seg_y.T, axis=1)
    
    # Juntam-se os dois vetores apenas num, e depois ordenam-se de forma a ter
    #os valores de x crescentes.
    if x_seen == xobs:
        vec = vec2
    elif m == 0:
        vec = vec1
    else:
        vec=np.array(np.append(vec1, vec2, axis=0))
    vec_ord=np.sort(vec,0)
    
    """
    Para cada ponto de intersesao o identifica-se qual a celula de onde a linha
    vem (e que portanto interseta a linha) e vai-se guardando a sequncia de
    clulas encontradas - vetor "cells".
    """
    
    cells=np.ceil(np.append(vec_ord[:,1:2],vec_ord[:,0:1],axis=1))
    
    """
    Obter a linha encontrada em formato matriz - comea-se com uma matriz de
    vazia (NaN) e alteram-se para 1 as clulas que pertencem  linha.
    """
    line=np.full((lin,col),np.nan)
    indice = cells[:,0] + (cells[:,1]-1)*lin
    indice = [int(x) for x in indice]
    
    line.ravel()[indice]=1
    
    """
    """
    K=(vec_ord[:,:1] - xobs) / vetor_diretor[0][0]

    Z=z_obs + np.dot(K,vetor_diretor[0][2])
    
    z_line=np.full((lin,col),np.nan)
    
    z_line.ravel()[indice]=np.ravel(Z)
    
    """
    """
    Dif=dem - z_line
    Dif[np.isnan(Dif)] = 0
    
    if np.sum(np.sum(Dif>0) > 0):
        vis=0
    else:
        vis=1
    
    return vis

In [10]:
# Test visibility

import datetime
vis = 0
time_a = datetime.datetime.now().replace(microsecond=0)
for row in range(lnhs):
    for col in range(cols):
        if np.isnan(np_lulc[row, col]) or np.isnan(np_dem[row, col]):
            continue
        else:
            if np_lulc[row, col] == 1:
                try:
                    __vis = check_visibility(
                        row_idx, col_idx, row, col, np_dem, lnhs, cols,
                    )
                except:
                    print(row, col)
                    __vis = 1
                
                if __vis == 1:
                    vis = __vis
                    break
    if vis:
        break
time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

0:11:07


d:\xpto\glass_env\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
d:\xpto\glass_env\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
vis = check_visibility(
    row_idx, col_idx, 338, 123, np_dem, lnhs, cols
)

In [11]:
print(vis)

1


In [ ]:
print(vec1.shape)
print(vec2.shape)

In [ ]:
print(np_dem)

In [ ]:
print(np_dem.shape)
print(np_lulc.shape)

In [ ]:
print(pnt_df)

In [ ]:
from PIL import Image

im = Image.Open(lulc)
lulc_class = np.array(im)